In [1]:
import pandas as pd
import numpy as np

# 1. Data Structures

## 1.1 Series 

pandas.Series is one-dimensional **ndarray** with **axis labels**. So it acts like an **ndarray** and a **dict**.

### 1.1.1 Create Series 

In [9]:
# Construct function
pd.Series(data=None, index=None, dtype=None, name=None, copy=False)
# Parameter data: array-like, Iterable, dict, scalar
# Parameter index: array-like or Index(1d)

# create Series from ndarray
s = pd.Series(np.random.randn(5))
print(s)

# create Series from array
s = pd.Series([2, 3, 1, 4, 5], index = ['a', 'b', 'c', 'd', 'e'])
print(s)

# create Series from dict
s = pd.Series({'a': 1.0, 'b': 2.0, 'c': 3.0}) # key as the index
print(s)
s = pd.Series({'a': 1.0, 'b': 2.0, 'c': 3.0}, index=['x', 'y', 'a', 'a']) # Non-unique index values are allowed.
# If an index is passed, the values in data corresponding to the labels in the index will be pulled out.
# Return NaN if no key corresponds to the index label.
print(s)

# create Series from scalar
s = pd.Series(5, index=['a', 'b', 'c'])
print(s)


0   -2.233972
1   -0.538305
2   -1.506726
3    0.279767
4    1.504825
dtype: float64
a    2
b    3
c    1
d    4
e    5
dtype: int64
a    1.0
b    2.0
c    3.0
dtype: float64
x    NaN
y    NaN
a    1.0
a    1.0
dtype: float64
a    5
b    5
c    5
dtype: int64


### 1.1.2 Series is ndarray-like 

Series acts very similarly to a ndarray and is a valid argument to most Numpy functions. And operations such as slicing will also slice the index.

In [19]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'], name='mySeries')
print(s[:3])
print(s['b':'d']) # different from number slicing, index slicing includes both start point and end point.
print(s[s > s.median()])

a    0.767637
b    0.008467
c   -0.536705
Name: mySeries, dtype: float64
b    0.008467
c   -0.536705
d    0.617515
Name: mySeries, dtype: float64
a    0.767637
d    0.617515
Name: mySeries, dtype: float64


Different from ndarray, operations between Series automatically align the data based on label. Thus, you can write computations without giving consideration to whether the Series involved have the same labels. If a label is not found in one Series or the other, the result will be marked as missing NaN.

In [23]:
s[1:] + s[:-1]

a         NaN
b    0.016933
c   -1.073411
d    1.235030
e         NaN
Name: mySeries, dtype: float64

While Series is ndarray-like, if you need an actual ndarray, use Series.to_numpy()

In [24]:
s.to_numpy()

array([ 0.76763684,  0.00846657, -0.53670536,  0.61751515, -0.41131902])

### 1.1.3 Series is dict-like 

In [25]:
print('a' in s)
print(s['a'])
print(s.get('a'))
print('f' in s)
# print(s['f']) # error
print(s.get('f'))
print(s.get('f', np.nan))

True
0.7676368427776733
0.7676368427776733
False
None
nan


## 1.2 DataFrame

pandas.DataFrame is a two-dimensional, size-multable, potentially heterogeneous tabular data. It contains labeled axes (rows and columns). Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

### 1.2.1 Create DataFrame 

In [35]:
# Construct function
pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
# Parameter data: ndarry(structured or homogeneous), Iterable, dict, or DataFrame

# create DataFrame from dict of Series
df = pd.DataFrame({'col1': pd.Series([1.0, 2.0, 3.0], index=['a', 'b', 'c']),
                  'col2': pd.Series([4.0, 5.0, 6.0], index=['a', 'b', 'd'])})
print(df)

# create DataFrame from dict of ndarrays/lists
df = pd.DataFrame({'col1':[1, 2, 3], 'col2':[4, 5, 6]}, index=['a', 'b', 'c'])
print(df)

# Optional
# create DataFrame from a list of dicts.
df = pd.DataFrame([{'a': 1, 'b': 2}, {'a': 4, 'b': 5, 'c': 6}])
print(df)

df = pd.DataFrame([{'a': 1, 'b': 2}, {'a': 4, 'b': 5, 'c': 6}], columns=['col1', 'col2'])
print(df)

# To summarize, keys in the dict data are for columns

   col1  col2
a   1.0   4.0
b   2.0   5.0
c   3.0   NaN
d   NaN   6.0
   col1  col2
a     1     4
b     2     5
c     3     6
   a  b    c
0  1  2  NaN
1  4  5  6.0
   col1  col2
0   NaN   NaN
1   NaN   NaN


### 1.2.2 Read and write DataFrame

 ### 1. ***pd.read_csv()***, read csv file & text file, return DataFrame

**Parameters:**  
**filepath_or_buffer**: filepath, str, url, or any object with a read() method (such as an open file or StringIO)

**sep / delimiter**: sep, default ','; delimiter, default None

**header**: default 'infer'. header=0: infer from first line. header=None: column names are passed. header='infer': if column names are passed, like header=None, else like header = 0. Note that this parameter ignores commented lines and empty lines if skip_blank_lines=True(default), so header=0 denotes the first line of data rather than the first line of the file.
**names**: array-like, default None. Column names.
**index_col**: int, str, sequence of int/str. Column(s) to use as row labels.

**usecols**: list-like or callable, default None. Return a subset of columns.
**skip_rows**: list-like or integer, default None. Line numbers to skip(0-indexed) or number of lines to skip (int) at the start of the file.
**nrows**: int, defatul None. Number of rows of file to read.

**true_values**: list, default None. Values to consider as True.
**false_values**: list, default None. Values to consider as False.
**na_values**: list and others, defaule None. Additional strings to recognize as NA/NaN.
**keep_default_na**: boolean, defatult True. Whether or not to include the default NaN values when parsing the data.

**parse_dates**: boolean or list of ints or names or list of lists or dict, default False.  
- If True: try parsing the index.  
- If [1, 2, 3]:try parsing columns 1,2,3 each as a separate data column. 
- If [[1, 3]]: combine columns 1 and 3 and parse as a single data column.
**date_format**: str or dict of column -> format, default None. If used in conjuction with parse_dates, will parse dates according to this format. For anything more complex, please read in as object and then apply to_datetime() as-needed.

**dtype**: indicate the date type for the whole DataFrame or individual columns.
**converters**: deal with columns with mixed types.

In [52]:
from io import BytesIO
data = b"word,length\n" b"Tr\xc3\xa4umen,7\n" b"Gr\xc3\xbc\xc3\x9fe,5"
df = pd.read_csv(BytesIO(data), encoding='utf-8') # By test, adding encoding='utf-8' or not won't affect the result
df

,word,length
0,Träumen,7
1,Grüße,5


### 2. to_csv(), Series/DataFrame instance method, store objects to csv/text file
**Parameters:**  
**path_or_buf**: a string path to the file or a file object.  
**sep**: Field delimiter, default ','.

In [53]:
from io import StringIO
with StringIO() as f:
    df.to_csv(f)
    f.seek(0)
    print(f.read())

,word,length
0,Träumen,7
1,Grüße,5



### 1.2.3 DataFrame basic functionality

In [59]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
df = pd.DataFrame(np.random.randn(8, 3), index=list('abcdefgh'), columns=["A", "B", "C"])

In [62]:
df.head() # display 5 head rows
df.head(10) # display 10 head rows
df.tail() # display 5 tail rows
df.tail(10) # display 10 tail rows

,A,B,C
a,0.306569,-0.709717,1.250645
b,1.363502,0.291905,1.128842
c,-0.845022,-1.460300,-1.807778
d,-0.035620,-0.916040,-1.883363
e,-1.623131,0.178064,1.719109
f,0.212458,0.586647,-1.203165
g,0.566506,0.656788,-0.768851
h,0.663305,-1.227375,-0.660401


In [63]:
df.shape

(8, 3)

In [65]:
df.index
df.columns

Index(['A', 'B', 'C'], dtype='object')

In [69]:
s.describe()
df.describe()

,A,B,C
count,8.000000,8.000000,8.000000
mean,0.076071,-0.325003,-0.278120
std,0.930649,0.847617,1.437503
min,-1.623131,-1.460300,-1.883363
25%,-0.237971,-0.993874,-1.354318
50%,0.259514,-0.265826,-0.714626
75%,0.590705,0.365591,1.159293
max,1.363502,0.656788,1.719109
